KODLAMA FAZI

Streamlit ile Hızlı Dashboard (MVP)

In [12]:
import streamlit as st
import pandas as pd
import plotly.express as px

# Veri yükleme
df = pd.read_csv("education_tracking_dataset_clean_en.csv")

# Dashboard başlık
st.title("🎓 Eğitim Takip Sistemi")

# 1. Öğrenci Performansı
st.header("Öğrenci Analizi")
selected_student = st.selectbox("Öğrenci Seçin", df["Student_Name"].unique())
student_data = df[df["Student_Name"] == selected_student]

# Not grafiği
fig_scores = px.bar(student_data, x="Course_Name", y="Score", color="Exam_Type",
                   title=f"{selected_student} Not Dağılımı")
st.plotly_chart(fig_scores)

# Devamsızlık durumu
attendance_status = student_data["Attendance_Status"].value_counts()
st.metric("Toplam Devamsızlık", f"{attendance_status.get('absent', 0)} kez")

# 2. Ders Bazlı Analiz
st.header("Ders Performansı")
course_stats = df.groupby("Course_Name").agg({"Score": "mean", "Attendance_Date": "count"})
st.dataframe(course_stats.style.background_gradient(cmap="Blues"))

# 3. Ham Veri Görüntüleme
if st.checkbox("Ham Veriyi Göster"):
    st.dataframe(df)

2025-08-06 11:32:19.143 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 11:32:19.144 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 11:32:19.149 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 11:32:19.151 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 11:32:19.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 11:32:19.156 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 11:32:19.163 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-06 11:32:19.168 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [14]:
!pip install streamlit plotly pandas
!streamlit run app.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


Flask + SQLite ile Tam Stack Web Uygulaması

In [16]:
!pip install Flask-SQLAlchemy

In [17]:
!pip install Flask-SQLAlchemy

In [19]:
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
import os

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///education.db'
db = SQLAlchemy(app)

# Veritabanı Modeli
class Student(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80))
    courses = db.relationship('Course', backref='student', lazy=True)

class Course(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(50))
    exam_type = db.Column(db.String(20))
    score = db.Column(db.Integer)
    student_id = db.Column(db.Integer, db.ForeignKey('student.id'))

# Ana sayfa
@app.route("/")
def dashboard():
    students = Student.query.all()
    return render_template("dashboard.html", students=students)

if __name__ == "__main__":
    # Create the database and load data before running the app
    with app.app_context():
        db.create_all()
        # Check if data has already been loaded to avoid duplicates on re-runs
        if not Student.query.first():
            df = pd.read_csv("education_tracking_dataset_clean_en.csv")
            for _, row in df.iterrows():
                # Use get_or_404 or handle the case where student_id might not exist
                # For simplicity, assuming Student_ID in CSV corresponds to a new student or existing one
                student = Student.query.filter_by(id=row["Student_ID"]).first()
                if not student:
                    student = Student(id=row["Student_ID"], name=row["Student_Name"])
                    db.session.add(student)
                    # Flush the session to make the new student id available
                    db.session.flush()

                course = Course(
                    name=row["Course_Name"],
                    exam_type=row["Exam_Type"],
                    score=row["Score"],
                    student_id=student.id # Use the student's id
                )
                db.session.add(course)
            db.session.commit()

    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
